In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/online-retail-data-v3/RetailDataIII.csv')
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data=data.dropna(subset=['CustomerID'])
data['BillDate']=pd.to_datetime(data['BillDate'])
data.tail()

In [ ]:
data.tail()

In [ ]:
data=data[data['Quota']>0]
data.info()

In [ ]:
def get_month(x): return dt.datetime(x.year,x.month,1)
data['billmonth']=data['BillDate'].apply(get_month)
data['cohortmonth']=data.groupby('CustomerID')['billmonth'].transform('min')
data.tail()

In [ ]:
def get_month_int(df,columns):
    year=df[columns].dt.year
    month=df[columns].dt.month
    day=df[columns].dt.day
    return year,month,day

billyear,billmonth,_=get_month_int(data,'billmonth')
cohortyear,cohortmonth,_=get_month_int(data,'cohortmonth')

conversion_year=billyear-cohortyear
conversion_month=billmonth-cohortmonth

data['cohortindex']=conversion_year*12+conversion_month+1

In [ ]:
cohortdata=data.groupby(['cohortindex','cohortmonth'])['CustomerID'].apply(pd.Series.nunique)
cohortdata=cohortdata.reset_index()
cohortdata=cohortdata.pivot(index='cohortmonth',columns='cohortindex',values='CustomerID')
cohortdata

In [ ]:
cohort_size=cohortdata.iloc[:,0]
retention=cohortdata.divide(cohort_size,axis=0)
retention

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(data=retention,annot=True,fmt='.0%',vmin=0.0,vmax=0.4,cmap='YlOrRd')
plt.show()

**RFM**

In [ ]:
data['total']=data['Quota']*data['Amount']
time_max=data['BillDate'].max()+dt.timedelta(days=1)

rfm=data.groupby('CustomerID').agg({'BillDate':lambda x:(time_max-x.max()).days,
                                    'Bill':'count',
                                    'total':'sum'})
rfm=rfm.rename(columns={'BillDate':'recently','Bill':'frequently','total':'money'})
rfm

In [ ]:
r_labels =range(4,0,-1)
f_labels=range(1,5)
m_labels=range(1,5)
r_quartiles = pd.qcut(rfm['recently'], q=4, labels = r_labels)
f_quartiles = pd.qcut(rfm['frequently'],q=4, labels = f_labels)
m_quartiles = pd.qcut(rfm['money'],q=4,labels = m_labels)
rfm = rfm.assign(R=r_quartiles,F=f_quartiles,M=m_quartiles)

def add_rfm(x) : return str(x['R']) + str(x['F']) + str(x['M'])
rfm['RFM_Segment'] = rfm.apply(add_rfm,axis=1)
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()